In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_path):
    if not os.path.isfile(zip_path):
        print(f"The file {zip_path} does not exist.")
        return

    if not os.path.exists(extract_path):
        os.makedirs(extract_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
        print(f"Files extracted to {extract_path}")

zip_file_path = 'images.zip'
extraction_path = 'images'

unzip_file(zip_file_path, extraction_path)


Files extracted to images


In [ ]:
import shutil
import os

# Define the path of the folder to be deleted
folder_path = 'images/train/disgusted'

# Check if the folder exists
if os.path.exists(folder_path):
    # Delete the folder
    shutil.rmtree(folder_path)
    print(f"The folder '{folder_path}' has been deleted.")
else:
    print(f"The folder '{folder_path}' does not exist.")


The folder 'images/train/disgusted' has been deleted.


In [ ]:
import os
import numpy as np
import shutil
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras import regularizers
from keras import layers, models
from keras.layers import Convolution2D as Conv2D
from keras.layers import MaxPooling2D as MaxPool2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [ ]:
train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					horizontal_flip=True,
					fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#using the inbuilt keras function we can load our data
#use the shuffle parameter to randomize and remove bias
#validation split divides the testing and training set
train = train_datagen.flow_from_directory(
					'images/train',
					color_mode='grayscale',
					target_size=(48, 48),
					batch_size=32,
					class_mode='categorical',
					shuffle=True)

Found 28273 images belonging to 6 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

validation = val_datagen.flow_from_directory(
    'images/test',
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    seed=42
)

Found 7067 images belonging to 6 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax'))


# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(48, 48, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.4))
# model.add(Dense(7))  # 7 classes
# model.add(Activation('softmax'))

In [ ]:
#we are telling the model to use categorical crossentropy due to our multi classification
#we are also telling the model to use the Adam algorithm
#finally we are telling the model to also give us accuracy rate through each epoch
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.optimizers import SGD

from keras.models import load_model

# new_model = load_model('emotion.h5')
# new_model.load_weights('weights.h5')





checkpoint = ModelCheckpoint("new_weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')


model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


model.fit(
    train,
    epochs=1,    #alter this number for more or less epochs
    validation_data=validation,
    callbacks=[checkpoint,early]
)

884/884 [==============================] - ETA: 0s - loss: 1.6940 - accuracy: 0.2879
Epoch 1: val_accuracy improved from -inf to 0.36550, saving model to new_weights.h5
884/884 [==============================] - 33s 32ms/step - loss: 1.6940 - accuracy: 0.2879 - val_loss: 1.5437 - val_accuracy: 0.3655


In [ ]:
model.save('emotion.h5')

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import load_model
import matplotlib.pyplot as plt
import itertools


def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

model = load_model('50pEmotion.h5')

# Predict the values from the test dataset
validation.reset()
Y_pred = model.predict(validation)
y_pred = np.argmax(Y_pred, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(validation.classes, y_pred)

# Plotting the confusion matrix
class_labels = list(validation.class_indices.keys())
plot_confusion_matrix(cm, class_labels, title='Confusion Matrix')
plt.show()

# Print classification report
print('Classification Report')
print(classification_report(validation.classes, y_pred, target_names=class_labels))


ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x78f6900c1810>.